In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf

# CNN metadata:

In [2]:
cle = pd.read_csv('cle_metadata_cnn.csv')
vir = pd.read_csv('vir_metadata_cnn.csv')
hun = pd.read_csv('hun_metadata_cnn.csv')
swi = pd.read_csv('swi_metadata_cnn.csv')

In [3]:
cle_train,cle_test = train_test_split(cle,test_size=0.33, random_state=42)
vir_train,vir_test = train_test_split(vir,test_size=0.33, random_state=42)
hun_train,hun_test = train_test_split(hun,test_size=0.33, random_state=42)
swi_train,swi_test = train_test_split(swi,test_size=0.33, random_state=42)

In [4]:
cle_train.to_csv('cle_metadata_cnn_train.csv',index = False)
cle_test.to_csv('cle_metadata_cnn_test.csv',index = False)

In [5]:
vir_train.to_csv('vir_metadata_cnn_train.csv',index = False)
vir_test.to_csv('vir_metadata_cnn_test.csv',index = False)

In [6]:
hun_train.to_csv('hun_metadata_cnn_train.csv',index = False)
hun_test.to_csv('hun_metadata_cnn_test.csv',index = False)

In [7]:
swi_train.to_csv('swi_metadata_cnn_train.csv',index = False)
swi_test.to_csv('swi_metadata_cnn_test.csv',index = False)

In [8]:
Train = pd.concat([cle_train,vir_train,hun_train,swi_train])
Test = pd.concat([cle_test,vir_test,hun_test,swi_test])

In [9]:
X_train = Train.iloc[:,:-1]
X_test = Test.iloc[:,:-1]

y_train = Train.iloc[:,-1]
y_test = Test.iloc[:,-1]

Y_train_binary = y_train.apply(lambda x: 1 if x > 0 else 0)
Y_test_binary = y_test.apply(lambda x: 1 if x > 0 else 0)

In [10]:
# function for deep learning testing
def Test(path_train,path_test,model_name):
    Train = pd.read_csv(path_train)
    Test = pd.read_csv(path_test)
    
    X_train = Train.iloc[:,:-1]
    Y_train = Train.iloc[:,-1]

    X_test = Test.iloc[:,:-1]
    Y_test = Test.iloc[:,-1]

    #binarize the target
    Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
    Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)

    model = tf.keras.models.load_model(model_name)

    Y_pred = model.predict(X_test).argmax(axis=1)
    
    cm = confusion_matrix(Y_pred, Y_test_binary)
    print(cm)
    print(classification_report(Y_test_binary, Y_pred, digits=4))
    
    mismatch = [i for i, (a,b) in enumerate(zip(Y_pred, Y_test_binary)) if a != b]
    print(mismatch)

# CNN

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define the model architecture
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(64,1)))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred, digits=4))

Epoch 1/1000
20/20 [==============================] - 4s 99ms/step - loss: 0.5181 - accuracy: 0.7707
Epoch 2/1000
20/20 [==============================] - 2s 86ms/step - loss: 0.3940 - accuracy: 0.8423
Epoch 3/1000
20/20 [==============================] - 2s 87ms/step - loss: 0.3571 - accuracy: 0.8650
Epoch 4/1000
20/20 [==============================] - 2s 88ms/step - loss: 0.3483 - accuracy: 0.8667
Epoch 5/1000
20/20 [==============================] - 2s 79ms/step - loss: 0.3744 - accuracy: 0.8618
Epoch 6/1000
20/20 [==============================] - 2s 78ms/step - loss: 0.3697 - accuracy: 0.8553
Epoch 7/1000
20/20 [==============================] - 2s 88ms/step - loss: 0.3521 - accuracy: 0.8813
Epoch 8/1000
20/20 [==============================] - 2s 98ms/step - loss: 0.3703 - accuracy: 0.8537
Epoch 9/1000
20/20 [==============================] - 2s 90ms/step - loss: 0.3405 - accuracy: 0.8683
Epoch 10/1000
20/20 [==============================] - 2s 88ms/step - loss: 0.3401 - accura

In [ ]:
model.save('../Models/Meta_Only/CNNMeta_CNN.h5')

# Test on each dataset

In [ ]:
print("Cle test")
path_train = 'cle_metadata_cnn_train.csv'
path_test = 'cle_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_CNN.h5'
Test(path_train,path_test,model)

print("==================")

print("vir test")
path_train = 'vir_metadata_cnn_train.csv'
path_test = 'vir_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_CNN.h5'
Test(path_train,path_test,model)

print("==================")
print("hun test")
path_train = 'hun_metadata_cnn_train.csv'
path_test = 'hun_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_CNN.h5'
Test(path_train,path_test,model)

print("==================")
print("swi test")
path_train = 'swi_metadata_cnn_train.csv'
path_test = 'swi_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_CNN.h5'
Test(path_train,path_test,model)

# DNN

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define the model architecture
model = Sequential()
model.add(Dense(64, input_shape=(64,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred, digits=4))

In [ ]:
model.save('../Models/Meta_Only/CNNMeta_DNN.h5')

# test on each dataset

In [ ]:
print("Cle test")
path_train = 'cle_metadata_cnn_train.csv'
path_test = 'cle_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_DNN.h5'
Test(path_train,path_test,model)

print("==================")

print("vir test")
path_train = 'vir_metadata_cnn_train.csv'
path_test = 'vir_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_DNN.h5'
Test(path_train,path_test,model)

print("==================")
print("hun test")
path_train = 'hun_metadata_cnn_train.csv'
path_test = 'hun_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_DNN.h5'
Test(path_train,path_test,model)

print("==================")
print("swi test")
path_train = 'swi_metadata_cnn_train.csv'
path_test = 'swi_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_DNN.h5'
Test(path_train,path_test,model)

# RNN

In [ ]:
from keras.layers import Dense, SimpleRNN

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define the model architecture
model = Sequential()
model.add(SimpleRNN(units=64, return_sequences=True, input_shape=(64, 1)))
model.add(SimpleRNN(units=32, return_sequences=True))
model.add(SimpleRNN(units=16))
model.add(Dense(units=2, activation='sigmoid'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred, digits=4))

In [ ]:
model.save('../Models/Meta_Only/CNNMeta_RNN.h5')

In [ ]:
print("Cle test")
path_train = 'cle_metadata_cnn_train.csv'
path_test = 'cle_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_RNN.h5'
Test(path_train,path_test,model)

print("==================")

print("vir test")
path_train = 'vir_metadata_cnn_train.csv'
path_test = 'vir_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_RNN.h5'
Test(path_train,path_test,model)

print("==================")
print("hun test")
path_train = 'hun_metadata_cnn_train.csv'
path_test = 'hun_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_RNN.h5'
Test(path_train,path_test,model)

print("==================")
print("swi test")
path_train = 'swi_metadata_cnn_train.csv'
path_test = 'swi_metadata_cnn_test.csv'
model = '../Models/Meta_Only/CNNMeta_RNN.h5'
Test(path_train,path_test,model)

# Machine Learning

# Decision Tree

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

In [ ]:
# decision tree
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(X_train, Y_train_binary)
Y_predictions = clf.predict(X_test)
cm = confusion_matrix(Y_predictions, Y_test_binary)
print("Confusion Matrix is:")
print(cm)
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements
print("Accuracy is : " + str(accuracy(cm)))
    
print("Report")
print(classification_report(Y_test_binary, Y_predictions))
    
#Visualization of the tree
plt.figure(figsize=(25,20))
tree.plot_tree(clf, filled=True, fontsize=10, max_depth= None,feature_names = list(X_train.columns), class_names = True)
plt.show()

In [ ]:
# save clf model
from joblib import dump, load
dump(clf, '../Models/Meta_only/CNNMeta_dt.joblib')

In [ ]:
# function for Decision tree, and random forest testing
def Test_DT(path_train,path_test,model_name):
    Train = pd.read_csv(path_train)
    Test = pd.read_csv(path_test)
    
    X_train = Train.iloc[:,:-1]
    Y_train = Train.iloc[:,-1]

    X_test = Test.iloc[:,:-1]
    Y_test = Test.iloc[:,-1]

    #binarize the target
    Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
    Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)

    model = load(model_name)

    Y_pred = model.predict(X_test)
    
    cm = confusion_matrix(Y_pred, Y_test_binary)
    print(cm)
    print(classification_report(Y_test_binary, Y_pred, digits=4))
    
    mismatch = [i for i, (a,b) in enumerate(zip(Y_pred, Y_test_binary)) if a != b]
    print(mismatch)

In [ ]:
#test on each dataset for decision tree
print("Cle test")
path_train = 'cle_metadata_cnn_train.csv'
path_test = 'cle_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_dt.joblib'
Test_DT(path_train,path_test,model)

print("==================")

print("vir test")
path_train = 'vir_metadata_cnn_train.csv'
path_test = 'vir_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_dt.joblib'
Test_DT(path_train,path_test,model)

print("==================")
print("hun test")
path_train = 'hun_metadata_cnn_train.csv'
path_test = 'hun_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_dt.joblib'
Test_DT(path_train,path_test,model)

print("==================")
print("swi test")
path_train = 'swi_metadata_cnn_train.csv'
path_test = 'swi_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_dt.joblib'
Test_DT(path_train,path_test,model)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, Y_train_binary)
Y_pred = classifier.predict(X_test)
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred))

In [ ]:
dump(classifier, '../Models/Meta_only/CNNMeta_rf.joblib')

In [ ]:
#test on each dataset for Random Forest
print("Cle test")
path_train = 'cle_metadata_cnn_train.csv'
path_test = 'cle_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_rf.joblib'
Test_DT(path_train,path_test,model)

print("==================")

print("vir test")
path_train = 'vir_metadata_cnn_train.csv'
path_test = 'vir_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_rf.joblib'
Test_DT(path_train,path_test,model)

print("==================")
print("hun test")
path_train = 'hun_metadata_cnn_train.csv'
path_test = 'hun_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_rf.joblib'
Test_DT(path_train,path_test,model)

print("==================")
print("swi test")
path_train = 'swi_metadata_cnn_train.csv'
path_test = 'swi_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_rf.joblib'
Test_DT(path_train,path_test,model)

# SVM

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC()
svc.fit(X_train, Y_train_binary.values.ravel())
y_pred = svc.predict(X_test)
print(confusion_matrix(Y_test_binary, y_pred))
print(classification_report(Y_test_binary, y_pred))

In [ ]:
import joblib
joblib.dump(svc, "../Models/Meta_only/CNNMeta_svm.pkl") 

In [ ]:
#test on each dataset for SVM
print("Cle test")
path_train = 'cle_metadata_cnn_train.csv'
path_test = 'cle_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_svm.pkl'
Test_DT(path_train,path_test,model)

print("==================")

print("vir test")
path_train = 'vir_metadata_cnn_train.csv'
path_test = 'vir_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_svm.pkl'
Test_DT(path_train,path_test,model)

print("==================")
print("hun test")
path_train = 'hun_metadata_cnn_train.csv'
path_test = 'hun_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_svm.pkl'
Test_DT(path_train,path_test,model)

print("==================")
print("swi test")
path_train = 'swi_metadata_cnn_train.csv'
path_test = 'swi_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_svm.pkl'
Test_DT(path_train,path_test,model)

# Bayesian Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(X_train, Y_train_binary)
Y_pred = classifier.predict(X_test)
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred))

dump(clf, '../Models/Meta_only/CNNMeta_NB.joblib')

In [ ]:
print("Cle test")
path_train = 'cle_metadata_cnn_train.csv'
path_test = 'cle_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_NB.joblib'
Test_DT(path_train,path_test,model)

print("==================")

print("vir test")
path_train = 'vir_metadata_cnn_train.csv'
path_test = 'vir_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_NB.joblib'
Test_DT(path_train,path_test,model)

print("==================")
print("hun test")
path_train = 'hun_metadata_cnn_train.csv'
path_test = 'hun_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_NB.joblib'
Test_DT(path_train,path_test,model)

print("==================")
print("swi test")
path_train = 'swi_metadata_cnn_train.csv'
path_test = 'swi_metadata_cnn_test.csv'
model = '../Models/Meta_only/CNNMeta_NB.joblib'
Test_DT(path_train,path_test,model)